# Routing

In [ ]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
from typing import Literal
from pydantic import Field, BaseModel
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate


class RouteQuery(BaseModel):
    """Route the user to the most relevant datasource."""
    
    datasource : Literal["python_docs", "js_docs", "golang_docs"] = Field(..., description="Given the user question choose which of the datasource will be the most relevant.")
    
# LLM
llm = ChatOpenAI(temperature=0)

# LLM with structured output parser
structured_llm = llm.with_structured_output(RouteQuery)

# Prompt
system = """
You are an expert in routing user's question to appropriate datasource.
Based on the programming language the question is referring to, route it to relevant datasource.  
"""

prompt = ChatPromptTemplate.from_messages(
   [
       ("system", system),
       ("human", "{question}")
   ]
)

# Router Chain
router = prompt | structured_llm  

C:\Users\Prateek Kumar\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_openai\chat_models\base.py:1686: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


In [9]:
question= """
Why this code is not working?

from langchina_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI()
parser = StrOutputParser()

template = '''You are an helpful assistant answer the question: '''
prompt = ChatPromptTemplate.from_template(
    template
)

chain =  prompt | llm | parser
"""

result = router.invoke({"question": question})

In [19]:
# Setting up route logic

def choose_route(result):
    if "python_docs" in result.datasource.lower():
        return "chain for python_docs"
    elif "js_docs" in result.datasource.lower():
        return "chain on js_docs"
    elif "golang_docs" in result.datasource.lower():
        return "chain on golang_docs"
    
from langchain_core.runnables import RunnableLambda

final_chain = router | RunnableLambda(choose_route)

In [20]:
result = final_chain.invoke({"question": question})

In [21]:
print(result)

chain for python_docs
